In [1]:
import pickle
import numpy as np
from dataset import Dataset, load_data, attention_init
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Masking, Dropout, LSTM, Bidirectional, Activation, Conv2D, Conv1D, MaxPool1D, AveragePooling1D, BatchNormalization
from keras.layers.merge import dot
from keras.models import Model, load_model
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras import backend as k

/home/fu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset_ger = 'berlin'
dataset_ita = 'dafex'
max_len = 516
features_number = 34
hidden_unit = 512
dropout_rate = 0.65
lstm_cells = 128
classes = 8
batch = 64
epochs = 3500

In [3]:
# DATA_ger, features_ger = load_data(dataset_ger)
# DATA_ita, features_ita = load_data(dataset_ita)

In [4]:
def data_concatenate(data1,data2,label1,label2,shuffle=False):
    mix = []
    mix_data = []
    mix_label = []
    for i in range(len(data1)):
        mix.append([data1[i], label1[i]])
    for i in range(len(data2)):
        mix.append([data2[i], label2[i]])
    if shuffle:
        np.random.shuffle(mix)
        for i in mix:
            mix_data.append(i[0])
            mix_label.append(i[1])
    else:
        for i in mix:
            mix_data.append(i[0])
            mix_label.append(i[1])
    return np.asarray(mix_data), np.asarray(mix_label)

In [5]:
def pltfunction(acc,loss,name):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    epoch = range(len(acc))
    plt.xlabel('epoch')
    plt.plot(epoch,acc,"x-",label=name+'acc')
    plt.plot(epoch,loss, "+-", label=name+'loss')
    plt.grid(True)
    plt.legend(loc=1)
    plt.savefig(name)
    plt.show()
    

In [28]:
def SER_mix_model(tra_data, tra_label, val_data, val_label, max_len,features_num, hidden_unit, dp_rate, lstm_cells, classes, epochs, batch_size):
#     label_test1 = to_categorical(label_test1, num_classes=classes)
#     label_test2 = to_categorical(label_test2, num_classes=classes)

#     tra_data, val_data, tra_label, val_label = train_test_split(data_test1, label_test1, test_size=0.3, random_state=886)

    u_train, u_val = attention_init(tra_data.shape[0], val_data.shape[0], 256, 1.0/256)
    
    model = load_model('./weights_blstm_mix_val_lossbest_ita_5cal.h5')
    
    model = Model(inputs=model.input,outputs=model.output)
    model.summary()
    optimizer = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)

    file_path = 'weights_blstm_mix_val_lossbest_ger_transfer_5cla' + '.h5'
    callback_list = [
                    EarlyStopping(
                        monitor='loss',
                        patience=150,
                        verbose=1,
                        mode='auto'
                    ),
                    ModelCheckpoint(
                        filepath=file_path,
                        monitor='val_loss',
                        save_best_only='True',
                        verbose=1,
                        mode='auto',
                        period=1
                    )
                    ]

    training = model.fit([u_train, tra_data], tra_label, batch_size=batch_size, epochs=epochs, verbose=1,
                             callbacks=callback_list, 
                             validation_data=([u_val,val_data], val_label))
    history = training.history
    acc = np.asarray(history['acc'])
    loss = np.asarray(history['loss'])
    pltfunction(acc,loss,"training_ger_tranfer")
    val_loss = np.asarray(history['val_loss'])
    val_acc = np.asarray(history['val_acc'])
    pltfunction(val_loss,val_acc,"testing_ger_transfer")

    acc_and_loss = np.column_stack((acc, loss, val_acc, val_loss))
    save_file_blstm = 'blstm_training_mix_val_lossbest_ger_tranfer_5cla' + '.csv'
    with open(save_file_blstm, 'wb'):
        np.savetxt(save_file_blstm, acc_and_loss)
    
#     u_test, _ = attention_init(x_test.shape[0], x_test.shape[0], 256, 1.0/256)
    score_test, accuracy_test = model.evaluate([u_val,val_data], val_label, batch_size=128, verbose=1)
#     score_2, accuracy_2 = model.evaluate([u_val2,x_test2], y_test2, batch_size=128, verbose=1)
    print('*******************************************************')
    print("Final test validation accuracy: %s" % accuracy_test)
#     print("Final test2 validation accuracy: %s" % accuracy_2)
    print('*******************************************************')
    
#     u_test_ger, u_test_ch = attention_init(val_data1.shape[0], val_data2.shape[0], 256, 1.0/256)
#     score_test_ger, accuracy_test_ger = model.evaluate([u_test_ger,val_data1], val_lable1, batch_size=128, verbose=1)
#     score_test_ita, accuracy_test_ita = model.evaluate([u_test_ch,val_data2], val_lable2, batch_size=128, verbose=1)
#     print('*******************************************************')
#     print("Final ger test validation accuracy: %s" % accuracy_test_ger)
#     print("Final ita test validation accuracy: %s" % accuracy_test_ita)
#     print('*******************************************************')
#     model_lossbest = load_model('./weights_blstm_mix_lossbest.h5')
#     score_2, accuracy_2 = model_lossbest.evaluate([u_test,x_test], y_label, batch_size=128, verbose=1)
#     print('*******************************************************')
#     print("Final test1 validation accuracy: %s" % accuracy_2)
# #     print("Final test2 validation accuracy: %s" % accuracy_2)
#     print('*******************************************************')
    
#     return {'loss': -accuracy, 'model': model}
    return model

In [29]:
ger_tra_label = pickle.load(open('tra_label_5cla_ger_labels.p', 'rb'))
ger_tra_feature = pickle.load(open('tra_data_5cla_ger_features.p', 'rb'))
ger_val_data=pickle.load(open('val_data_5cla_ger_features.p','rb'))
ger_val_label=pickle.load(open('val_label_5cla_ger_labels.p','rb'))

m = SER_mix_model(tra_data=ger_tra_feature, tra_label=ger_tra_label, val_data=ger_val_data, val_label=ger_val_label, max_len=max_len, features_num=features_number, hidden_unit=hidden_unit, dp_rate=dropout_rate,
          lstm_cells=lstm_cells, classes=classes, epochs=epochs, batch_size=batch)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 516, 34)      0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 514, 512)     52736       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 514, 1024)    525312      conv1d_4[0][0]                   
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 512, 2048)    6293504     conv1d_5[0][0]                   
__________________________________________________________________________________________________
max_poolin


Epoch 00018: val_loss improved from 1.23299 to 1.21395, saving model to weights_blstm_mix_val_lossbest_ger_transfer_5cla.h5
Epoch 19/3500
285/285 [==============================] - 5s 19ms/step - loss: 1.3130 - acc: 0.5193 - val_loss: 1.2001 - val_acc: 0.5285

Epoch 00019: val_loss improved from 1.21395 to 1.20010, saving model to weights_blstm_mix_val_lossbest_ger_transfer_5cla.h5
Epoch 20/3500
285/285 [==============================] - 5s 19ms/step - loss: 1.3550 - acc: 0.4702 - val_loss: 1.1956 - val_acc: 0.5285

Epoch 00020: val_loss improved from 1.20010 to 1.19562, saving model to weights_blstm_mix_val_lossbest_ger_transfer_5cla.h5
Epoch 21/3500
285/285 [==============================] - 5s 19ms/step - loss: 1.3064 - acc: 0.5053 - val_loss: 1.1837 - val_acc: 0.5285

Epoch 00021: val_loss improved from 1.19562 to 1.18369, saving model to weights_blstm_mix_val_lossbest_ger_transfer_5cla.h5
Epoch 22/3500
285/285 [==============================] - 5s 19ms/step - loss: 1.2895 - acc: 

285/285 [==============================] - 5s 19ms/step - loss: 1.0607 - acc: 0.5789 - val_loss: 1.0080 - val_acc: 0.5935

Epoch 00055: val_loss did not improve from 0.97629
Epoch 56/3500
285/285 [==============================] - 5s 19ms/step - loss: 1.0986 - acc: 0.5544 - val_loss: 1.1828 - val_acc: 0.5447

Epoch 00056: val_loss did not improve from 0.97629
Epoch 57/3500
285/285 [==============================] - 5s 18ms/step - loss: 1.1104 - acc: 0.5579 - val_loss: 1.0110 - val_acc: 0.5854

Epoch 00057: val_loss did not improve from 0.97629
Epoch 58/3500
285/285 [==============================] - 5s 19ms/step - loss: 1.0694 - acc: 0.5509 - val_loss: 0.9797 - val_acc: 0.5935

Epoch 00058: val_loss did not improve from 0.97629
Epoch 59/3500
285/285 [==============================] - 5s 18ms/step - loss: 1.0580 - acc: 0.5614 - val_loss: 1.0088 - val_acc: 0.5854

Epoch 00059: val_loss did not improve from 0.97629
Epoch 60/3500
285/285 [==============================] - 5s 18ms/step - lo


Epoch 00095: val_loss did not improve from 0.94023
Epoch 96/3500
285/285 [==============================] - 5s 19ms/step - loss: 1.0295 - acc: 0.5684 - val_loss: 0.9338 - val_acc: 0.5935

Epoch 00096: val_loss improved from 0.94023 to 0.93382, saving model to weights_blstm_mix_val_lossbest_ger_transfer_5cla.h5
Epoch 97/3500
285/285 [==============================] - 5s 18ms/step - loss: 1.0083 - acc: 0.5895 - val_loss: 0.9794 - val_acc: 0.5610

Epoch 00097: val_loss did not improve from 0.93382
Epoch 98/3500
285/285 [==============================] - 5s 19ms/step - loss: 1.0154 - acc: 0.5825 - val_loss: 0.9508 - val_acc: 0.5935

Epoch 00098: val_loss did not improve from 0.93382
Epoch 99/3500
285/285 [==============================] - 5s 18ms/step - loss: 1.0277 - acc: 0.5684 - val_loss: 0.9287 - val_acc: 0.5935

Epoch 00099: val_loss improved from 0.93382 to 0.92874, saving model to weights_blstm_mix_val_lossbest_ger_transfer_5cla.h5
Epoch 100/3500
285/285 [==========================

285/285 [==============================] - 5s 19ms/step - loss: 0.9752 - acc: 0.6140 - val_loss: 0.9640 - val_acc: 0.5772

Epoch 00136: val_loss did not improve from 0.91036
Epoch 137/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.9985 - acc: 0.6140 - val_loss: 1.1761 - val_acc: 0.4878

Epoch 00137: val_loss did not improve from 0.91036
Epoch 138/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.9963 - acc: 0.5860 - val_loss: 1.0312 - val_acc: 0.5854

Epoch 00138: val_loss did not improve from 0.91036
Epoch 139/3500
285/285 [==============================] - 5s 18ms/step - loss: 0.9859 - acc: 0.5895 - val_loss: 0.9095 - val_acc: 0.5935

Epoch 00139: val_loss improved from 0.91036 to 0.90948, saving model to weights_blstm_mix_val_lossbest_ger_transfer_5cla.h5
Epoch 140/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.9550 - acc: 0.5930 - val_loss: 0.9095 - val_acc: 0.5935

Epoch 00140: val_loss did not improve from 0.909

...

285/285 [==============================] - 5s 19ms/step - loss: 0.5993 - acc: 0.7684 - val_loss: 1.1393 - val_acc: 0.5935

Epoch 01221: val_loss did not improve from 0.67503
Epoch 1222/3500
285/285 [==============================] - 5s 18ms/step - loss: 0.6794 - acc: 0.7263 - val_loss: 0.8942 - val_acc: 0.6179

Epoch 01222: val_loss did not improve from 0.67503
Epoch 1223/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.6118 - acc: 0.7544 - val_loss: 0.9392 - val_acc: 0.6179

Epoch 01223: val_loss did not improve from 0.67503
Epoch 1224/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.5961 - acc: 0.7789 - val_loss: 1.2158 - val_acc: 0.6260

Epoch 01224: val_loss did not improve from 0.67503
Epoch 1225/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.5957 - acc: 0.7719 - val_loss: 0.7179 - val_acc: 0.6748

Epoch 01225: val_loss did not improve from 0.67503
Epoch 1226/3500
285/285 [==============================] - 5s 18ms

285/285 [==============================] - 5s 18ms/step - loss: 0.5791 - acc: 0.7649 - val_loss: 0.6906 - val_acc: 0.6911

Epoch 01264: val_loss did not improve from 0.67503
Epoch 1265/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.5698 - acc: 0.7614 - val_loss: 0.8977 - val_acc: 0.6098

Epoch 01265: val_loss did not improve from 0.67503
Epoch 1266/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.6321 - acc: 0.7614 - val_loss: 1.0122 - val_acc: 0.6423

Epoch 01266: val_loss did not improve from 0.67503
Epoch 1267/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.5926 - acc: 0.7719 - val_loss: 0.8494 - val_acc: 0.6341

Epoch 01267: val_loss did not improve from 0.67503
Epoch 1268/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.5951 - acc: 0.7860 - val_loss: 0.7206 - val_acc: 0.6911

Epoch 01268: val_loss did not improve from 0.67503
Epoch 1269/3500
285/285 [==============================] - 5s 18ms

285/285 [==============================] - 5s 18ms/step - loss: 0.5747 - acc: 0.8000 - val_loss: 0.8753 - val_acc: 0.6341

Epoch 01307: val_loss did not improve from 0.67415
Epoch 1308/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.6291 - acc: 0.7333 - val_loss: 0.7500 - val_acc: 0.6667

Epoch 01308: val_loss did not improve from 0.67415
Epoch 1309/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.6291 - acc: 0.7474 - val_loss: 0.8539 - val_acc: 0.6585

Epoch 01309: val_loss did not improve from 0.67415
Epoch 1310/3500
285/285 [==============================] - 5s 18ms/step - loss: 0.6080 - acc: 0.7684 - val_loss: 0.7466 - val_acc: 0.6504

Epoch 01310: val_loss did not improve from 0.67415
Epoch 1311/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.6255 - acc: 0.7509 - val_loss: 0.6953 - val_acc: 0.6829

Epoch 01311: val_loss did not improve from 0.67415
Epoch 1312/3500
285/285 [==============================] - 5s 18ms

285/285 [==============================] - 5s 18ms/step - loss: 0.5944 - acc: 0.7368 - val_loss: 1.2807 - val_acc: 0.5041

Epoch 01350: val_loss did not improve from 0.67415
Epoch 1351/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.6532 - acc: 0.7263 - val_loss: 1.4681 - val_acc: 0.5203

Epoch 01351: val_loss did not improve from 0.67415
Epoch 1352/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.6001 - acc: 0.7930 - val_loss: 0.7788 - val_acc: 0.6911

Epoch 01352: val_loss did not improve from 0.67415
Epoch 1353/3500
285/285 [==============================] - 5s 19ms/step - loss: 0.6304 - acc: 0.7509 - val_loss: 0.7430 - val_acc: 0.6992

Epoch 01353: val_loss did not improve from 0.67415
Epoch 1354/3500
285/285 [==============================] - 5s 18ms/step - loss: 0.5443 - acc: 0.7965 - val_loss: 0.9765 - val_acc: 0.5691

Epoch 01354: val_loss did not improve from 0.67415
Epoch 1355/3500
285/285 [==============================] - 5s 18ms

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
def SER_mix_model(tra_data, val_data, tra_label, val_label, max_len,features_num, hidden_unit, dp_rate, lstm_cells, classes, epochs, batch_size):
# def SER_mix_model(data_test1, label_test1, data_test2, label_test2, max_len,features_num, hidden_unit, dp_rate, lstm_cells, classes, epochs, batch_size):
#     label_test1 = to_categorical(label_test1, num_classes=classes)
#     label_test2 = to_categorical(label_test2, num_classes=classes)

#     tra_data, val_data, tra_label, val_label = train_test_split(data_test2, label_test2, test_size=0.3, random_state=886)

    u_train, u_val = attention_init(tra_data.shape[0], val_data.shape[0], 256, 1.0/256)
    
    model = load_model('./weights_blstm_mix_val_lossbest_ger_5cal.h5')
    
    model = Model(inputs=model.input,outputs=model.output)
    model.summary()
    optimizer = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)

    file_path = 'weights_blstm_mix_val_lossbest_ita_transfer_5cla' + '.h5'
    callback_list = [
                    EarlyStopping(
                        monitor='loss',
                        patience=150,
                        verbose=1,
                        mode='auto'
                    ),
                    ModelCheckpoint(
                        filepath=file_path,
                        monitor='val_loss',
                        save_best_only='True',
                        verbose=1,
                        mode='auto',
                        period=1
                    )
                    ]

    training = model.fit([u_train, tra_data], tra_label, batch_size=batch_size, epochs=epochs, verbose=1,
                             callbacks=callback_list, 
                             validation_data=([u_val,val_data], val_label))
    history = training.history
    acc = np.asarray(history['acc'])
    loss = np.asarray(history['loss'])
    pltfunction(acc,loss,"training_ita_tranfer")
    val_loss = np.asarray(history['val_loss'])
    val_acc = np.asarray(history['val_acc'])
    pltfunction(val_loss,val_acc,"testing_ita_transfer")

    acc_and_loss = np.column_stack((acc, loss, val_acc, val_loss))
    save_file_blstm = 'blstm_training_mix_val_lossbest_ita_tranfer' + '.csv'
    with open(save_file_blstm, 'wb'):
        np.savetxt(save_file_blstm, acc_and_loss)
    
#     u_test, _ = attention_init(x_test.shape[0], x_test.shape[0], 256, 1.0/256)
    score_test, accuracy_test = model.evaluate([u_val,val_data], val_label, batch_size=128, verbose=1)
#     score_2, accuracy_2 = model.evaluate([u_val2,x_test2], y_test2, batch_size=128, verbose=1)
    print('*******************************************************')
    print("Final test validation accuracy: %s" % accuracy_test)
#     print("Final test2 validation accuracy: %s" % accuracy_2)
    print('*******************************************************')
    
#     u_test_ger, u_test_ch = attention_init(val_data1.shape[0], val_data2.shape[0], 256, 1.0/256)
#     score_test_ger, accuracy_test_ger = model.evaluate([u_test_ger,val_data1], val_lable1, batch_size=128, verbose=1)
#     score_test_ita, accuracy_test_ita = model.evaluate([u_test_ch,val_data2], val_lable2, batch_size=128, verbose=1)
#     print('*******************************************************')
#     print("Final ger test validation accuracy: %s" % accuracy_test_ger)
#     print("Final ita test validation accuracy: %s" % accuracy_test_ita)
#     print('*******************************************************')
#     model_lossbest = load_model('./weights_blstm_mix_lossbest.h5')
#     score_2, accuracy_2 = model_lossbest.evaluate([u_test,x_test], y_label, batch_size=128, verbose=1)
#     print('*******************************************************')
#     print("Final test1 validation accuracy: %s" % accuracy_2)
# #     print("Final test2 validation accuracy: %s" % accuracy_2)
#     print('*******************************************************')
    
#     return {'loss': -accuracy, 'model': model}
    return model

In [31]:
ita_tra_label = pickle.load(open('tra_label_5cla_ita_labels.p', 'rb'))
ita_tra_feature = pickle.load(open('tra_data_5cla_ita_features.p', 'rb'))
ita_val_data=pickle.load(open('val_data_5cla_ita_features.p','rb'))
ita_val_label=pickle.load(open('val_label_5cla_ita_labels.p','rb'))

m = SER_mix_model(tra_data=ita_tra_feature, tra_label=ita_tra_label, val_data=ita_val_data, val_label=ita_val_label, max_len=max_len, features_num=features_number, hidden_unit=hidden_unit, dp_rate=dropout_rate,
          lstm_cells=lstm_cells, classes=classes, epochs=epochs, batch_size=batch)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 516, 34)      0                                            
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 514, 512)     52736       input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 514, 1024)    525312      conv1d_16[0][0]                  
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 512, 2048)    6293504     conv1d_17[0][0]                  
__________________________________________________________________________________________________
max_poolin

336/336 [==============================] - 6s 19ms/step - loss: 1.2944 - acc: 0.4583 - val_loss: 1.5632 - val_acc: 0.3889

Epoch 00030: val_loss did not improve from 1.48460
Epoch 31/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.2879 - acc: 0.5000 - val_loss: 1.5007 - val_acc: 0.3472

Epoch 00031: val_loss did not improve from 1.48460
Epoch 32/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.2937 - acc: 0.4702 - val_loss: 1.4840 - val_acc: 0.3681

Epoch 00032: val_loss improved from 1.48460 to 1.48400, saving model to weights_blstm_mix_val_lossbest_ita_transfer_5cla.h5
Epoch 33/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.3128 - acc: 0.4762 - val_loss: 1.4563 - val_acc: 0.3819

Epoch 00033: val_loss improved from 1.48400 to 1.45631, saving model to weights_blstm_mix_val_lossbest_ita_transfer_5cla.h5
Epoch 34/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.2613 - acc: 0.4881 - val_loss: 1.485

336/336 [==============================] - 6s 19ms/step - loss: 1.1741 - acc: 0.4940 - val_loss: 2.2485 - val_acc: 0.2222

Epoch 00070: val_loss did not improve from 1.34760
Epoch 71/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.1846 - acc: 0.5060 - val_loss: 4.1727 - val_acc: 0.2431

Epoch 00071: val_loss did not improve from 1.34760
Epoch 72/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.1965 - acc: 0.5119 - val_loss: 2.7335 - val_acc: 0.2986

Epoch 00072: val_loss did not improve from 1.34760
Epoch 73/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.1963 - acc: 0.5060 - val_loss: 2.1035 - val_acc: 0.3542

Epoch 00073: val_loss did not improve from 1.34760
Epoch 74/3500
336/336 [==============================] - 7s 19ms/step - loss: 1.2052 - acc: 0.5179 - val_loss: 1.4155 - val_acc: 0.3958

Epoch 00074: val_loss did not improve from 1.34760
Epoch 75/3500
336/336 [==============================] - 6s 19ms/step - lo


Epoch 00111: val_loss did not improve from 1.29128
Epoch 112/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.1095 - acc: 0.5565 - val_loss: 1.2926 - val_acc: 0.4028

Epoch 00112: val_loss did not improve from 1.29128
Epoch 113/3500
336/336 [==============================] - 7s 19ms/step - loss: 1.1188 - acc: 0.5357 - val_loss: 1.4617 - val_acc: 0.4028

Epoch 00113: val_loss did not improve from 1.29128
Epoch 114/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.1437 - acc: 0.5327 - val_loss: 1.3020 - val_acc: 0.4028

Epoch 00114: val_loss did not improve from 1.29128
Epoch 115/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.1260 - acc: 0.5357 - val_loss: 1.3438 - val_acc: 0.4097

Epoch 00115: val_loss did not improve from 1.29128
Epoch 116/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.1187 - acc: 0.5298 - val_loss: 1.3169 - val_acc: 0.4097

Epoch 00116: val_loss did not improve from 1.29128
Epo

336/336 [==============================] - 6s 19ms/step - loss: 1.0910 - acc: 0.5387 - val_loss: 1.2576 - val_acc: 0.4236

Epoch 00154: val_loss did not improve from 1.22387
Epoch 155/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.0924 - acc: 0.5565 - val_loss: 1.2601 - val_acc: 0.4306

Epoch 00155: val_loss did not improve from 1.22387
Epoch 156/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.0934 - acc: 0.5685 - val_loss: 1.2348 - val_acc: 0.4375

Epoch 00156: val_loss did not improve from 1.22387
Epoch 157/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.0923 - acc: 0.5625 - val_loss: 1.4077 - val_acc: 0.4028

Epoch 00157: val_loss did not improve from 1.22387
Epoch 158/3500
336/336 [==============================] - 6s 19ms/step - loss: 1.0939 - acc: 0.5446 - val_loss: 1.6543 - val_acc: 0.3264

Epoch 00158: val_loss did not improve from 1.22387
Epoch 159/3500
336/336 [==============================] - 7s 20ms/step

...


Epoch 00807: val_loss did not improve from 1.04688
Epoch 808/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.8201 - acc: 0.6726 - val_loss: 1.2943 - val_acc: 0.5278

Epoch 00808: val_loss did not improve from 1.04688
Epoch 809/3500
336/336 [==============================] - 7s 20ms/step - loss: 0.8378 - acc: 0.6488 - val_loss: 1.2671 - val_acc: 0.5000

Epoch 00809: val_loss did not improve from 1.04688
Epoch 810/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.8213 - acc: 0.6637 - val_loss: 1.3293 - val_acc: 0.5000

Epoch 00810: val_loss did not improve from 1.04688
Epoch 811/3500
336/336 [==============================] - 7s 19ms/step - loss: 0.8341 - acc: 0.6577 - val_loss: 3.2188 - val_acc: 0.3056

Epoch 00811: val_loss did not improve from 1.04688
Epoch 812/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.8260 - acc: 0.6548 - val_loss: 1.0638 - val_acc: 0.5556

Epoch 00812: val_loss did not improve from 1.04688
Epo

336/336 [==============================] - 6s 19ms/step - loss: 0.8855 - acc: 0.6220 - val_loss: 1.6545 - val_acc: 0.4306

Epoch 00851: val_loss did not improve from 1.04688
Epoch 852/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.8494 - acc: 0.6399 - val_loss: 1.5720 - val_acc: 0.4514

Epoch 00852: val_loss did not improve from 1.04688
Epoch 853/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.8422 - acc: 0.6577 - val_loss: 3.9465 - val_acc: 0.3333

Epoch 00853: val_loss did not improve from 1.04688
Epoch 854/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.8363 - acc: 0.6607 - val_loss: 3.1685 - val_acc: 0.3681

Epoch 00854: val_loss did not improve from 1.04688
Epoch 855/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.8665 - acc: 0.6399 - val_loss: 1.7578 - val_acc: 0.4236

Epoch 00855: val_loss did not improve from 1.04688
Epoch 856/3500
336/336 [==============================] - 6s 19ms/step

336/336 [==============================] - 7s 19ms/step - loss: 0.7920 - acc: 0.6518 - val_loss: 1.2707 - val_acc: 0.4931

Epoch 00894: val_loss did not improve from 1.04688
Epoch 895/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.7616 - acc: 0.6696 - val_loss: 1.1407 - val_acc: 0.5278

Epoch 00895: val_loss did not improve from 1.04688
Epoch 896/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.8033 - acc: 0.6548 - val_loss: 1.2878 - val_acc: 0.5069

Epoch 00896: val_loss did not improve from 1.04688
Epoch 897/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.7902 - acc: 0.6488 - val_loss: 2.2495 - val_acc: 0.4375

Epoch 00897: val_loss did not improve from 1.04688
Epoch 898/3500
336/336 [==============================] - 6s 19ms/step - loss: 0.7844 - acc: 0.6845 - val_loss: 2.0869 - val_acc: 0.4167

Epoch 00898: val_loss did not improve from 1.04688
Epoch 899/3500
336/336 [==============================] - 6s 19ms/step

336/336 [==============================] - 6s 19ms/step - loss: 0.7939 - acc: 0.6458 - val_loss: 1.4802 - val_acc: 0.4722

Epoch 00937: val_loss did not improve from 1.04627
Epoch 938/3500
320/336 [===========================>..] - ETA: 0s - loss: 0.7979 - acc: 0.6781

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
def test_model(val_data, val_label, model_path):
#     val_label = to_categorical(val_label, num_classes=classes)
    
#     x_test, val_data, y_label, val_label = train_test_split(data, label, test_size=0.3, random_state=886)

    u_val, _ = attention_init(val_data.shape[0], val_data.shape[0], 256, 1.0/256)
    model = load_model(model_path)

    score, accuracy = model.evaluate([u_val,val_data], val_label, batch_size=128, verbose=1)

    print('*******************************************************')
    print("Final test validation accuracy: %s" % accuracy)
    print('*******************************************************')
   

In [33]:
ger_data=pickle.load(open('val_data_5cla_ger_features.p','rb'))
ger_label=pickle.load(open('val_label_5cla_ger_labels.p','rb'))
ita_data=pickle.load(open('val_data_5cla_ita_features.p','rb'))
ita_label=pickle.load(open('val_label_5cla_ita_labels.p','rb'))

In [35]:
path_ger='./weights_blstm_mix_val_lossbest_ger_transfer_5cla.h5'
path_ita='./weights_blstm_mix_val_lossbest_ita_transfer_5cla.h5'
test_model(val_data=ger_data,val_label=ger_label,model_path=path_ita)
test_model(val_data=ita_data,val_label=ita_label,model_path=path_ger)

123/123 [==============================] - 6s 49ms/step
*******************************************************
Final test validation accuracy: 0.35772356390953064
*******************************************************
144/144 [==============================] - 7s 47ms/step
*******************************************************
Final test validation accuracy: 0.2152777777777778
*******************************************************
